# Polynomial Interpolation

In this notebook, we explore **polynomial interpolation**, a fundamental tool in numerical analysis. Given a set of data points $(x_i, y_i)$, the goal is to find a polynomial that passes through all of them exactly. This concept is widely used in data fitting, computer graphics, and solving differential equations.

This notebook
- Introduces the **theory** behind interpolation.
- Visualizes interpolation with increasing polynomial degrees.
- Examines phenomena such as **Runge’s phenomenon**.
- Experiments with different configurations of interpolation nodes.



## Polynomial Interpolation

Let $f(x)$ be a function defined on an interval $[a, b]$, and let $\{x_i\}_{i=0}^n$ be a set of $n+1$ interpolation nodes. The interpolating polynomial $P_n(x)$ of degree at most $n$ is uniquely determined by the condition:

$$
P_n(x_i) = f(x_i) \quad \text{for } i = 0, 1, ..., n.
$$

The polynomial is constructed using the **Vandermonde matrix**:

$$
V = \begin{bmatrix}
x_0^n & x_0^{n-1} & \dots & 1 \\
x_1^n & x_1^{n-1} & \dots & 1 \\
\vdots & \vdots & \ddots & \vdots \\
x_n^n & x_n^{n-1} & \dots & 1 \\
\end{bmatrix}
$$

Solving the linear system $V \cdot \vec{c} = \vec{y}$ gives the coefficients of the interpolating polynomial.


This cell allows you to enter a symbolic function and visualize its interpolation polynomial of a chosen degree, computed over a specified domain. Two types of interpolation nodes are supported:

- **Equally Spaced Nodes:** Nodes are linearly spaced in the domain.
- **Chebyshev Nodes:** Specially distributed nodes that reduce error and oscillations at the edges.

- **Left Plot:** Shows the original function $f(x)$, the interpolating polynomial $P_n(x)$, and interpolation nodes.
- **Right Plot:** Displays the pointwise interpolation error $|f(x) - P_n(x)|$.

### Instructions

1. Enter a symbolic function (e.g., `sin(x)` or `exp(-x**2)`).
2. Adjust the polynomial degree using the slider.
3. Change the interpolation domain.
4. Choose between `Equally Spaced` and `Chebyshev` nodes.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sympy as sp
from IPython.display import display, Math

function_guide = widgets.HTML(value="""
<h4>Supported Functions</h4>
<p>Enter expressions like:</p>
<ul>
<li><b>Basic:</b> <code>sin(x), cos(x), tan(x), exp(x), log(x)</code></li>
<li><b>Advanced:</b> <code>abs(x), sqrt(x), exp(-x**2)</code></li>
</ul>
<b>Examples:</b>
<code>np.sin(x) + x**2</code>, <code>np.exp(-x**2)</code>, <code>np.log(abs(x+1))</code>
""")

def interpolate_and_plot(f_str, degree, domain, node_type):
    x = sp.symbols('x')
    try:
        f_expr = sp.sympify(f_str)
        f = sp.lambdify(x, f_expr, modules=['numpy'])
    except:
        print("Invalid function! Try again.")
        return

    n_points = degree + 1
    if node_type == 'Equally Spaced':
        x_nodes = np.linspace(domain[0], domain[1], n_points)
    else:  # Chebyshev Nodes
        k = np.arange(n_points)
        cheb = np.cos((2 * k + 1) * np.pi / (2 * n_points))
        x_nodes = 0.5 * (domain[0] + domain[1]) + 0.5 * (domain[1] - domain[0]) * cheb

    y_nodes = f(x_nodes)

    V = np.vander(x_nodes, n_points)
    coeffs = np.linalg.solve(V, y_nodes)

    poly_expr = sum(coeffs[i] * x**(degree - i) for i in range(n_points))
    poly_expr = sp.expand(poly_expr)

    p = np.poly1d(coeffs)

    x_plot = np.linspace(domain[0], domain[1], 1000)
    y_plot = f(x_plot)
    y_poly = p(x_plot)

    error = np.abs(y_plot - y_poly)
    max_error = np.max(error)
    rms_error = np.sqrt(np.mean(error**2))

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].plot(x_plot, y_plot, color="blue", lw=2, label="Original Function")
    ax[0].plot(x_plot, y_poly, color="green", lw=2, linestyle="--", label=f"Degree {degree} Polynomial")
    ax[0].scatter(x_nodes, y_nodes, color='red', edgecolor='black', s=50, label="Interpolation Nodes")

    ax[0].set_title("Polynomial Interpolation")
    ax[0].set_xlabel("x")
    ax[0].set_ylabel("y")
    ax[0].legend()
    ax[0].grid(True)

    ax[1].plot(x_plot, error, color='purple', lw=2, label='Absolute Error')
    ax[1].set_title("Interpolation Error")
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("|f(x) - P(x)|")
    ax[1].legend()
    ax[1].grid(True)

    ax[1].text(domain[0] + 0.1, max(error) * 0.8, f"Max Error: {max_error:.4e}\nRMS Error: {rms_error:.4e}",
                fontsize=10, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))

    if node_type == 'Equally Spaced' and degree > 10:
        ax[0].text(domain[0] + 0.2, max(y_plot) * 0.8, "High oscillations at edges!", fontsize=10, color="red",
                   bbox=dict(facecolor="white", edgecolor="red", boxstyle="round,pad=0.5"))
        ax[1].axvspan(domain[0], domain[0] + 0.3, color="red", alpha=0.2)
        ax[1].axvspan(domain[1] - 0.3, domain[1], color="red", alpha=0.2)
        ax[1].text(domain[1] - 0.25, max(error) * 0.5, "Edge Instability!", fontsize=12, fontweight="bold", color="red", rotation=90,
           bbox=dict(facecolor="white", edgecolor="black", linewidth=2, boxstyle="round,pad=0.5"))

    elif node_type == 'Chebyshev' and degree > 10:
        ax[0].text(domain[0] + 0.2, max(y_plot) * 0.8, "Chebyshev reduces edge oscillation!", fontsize=10, color="black",
                   bbox=dict(facecolor="lightgreen", edgecolor="black", boxstyle="round,pad=0.5"))

    plt.tight_layout()
    plt.show()

    display(Math(rf"P(x) = {sp.latex(poly_expr)}"))

iw = widgets.interactive(interpolate_and_plot,
                          f_str=widgets.Text(value='sin(x)', description='Function:'),
                          degree=widgets.IntSlider(min=1, max=20, value=3, description='Degree:'),
                          domain=widgets.FloatRangeSlider(value=[-3, 3], min=-10, max=10, description='Domain:'),
                          node_type=widgets.ToggleButtons(options=['Equally Spaced', 'Chebyshev'], description='Nodes:'))
ui = widgets.VBox([
    function_guide,
    iw
])

display(ui)




## Runge’s Phenomenon

This cell explores one of the classical challenges in polynomial interpolation: **Runge’s phenomenon**. The **Runge function** is defined as:

$$
f(x) = \frac{1}{1 + 25x^2}
$$

Although smooth, it demonstrates that increasing the polynomial degree with **equally spaced nodes** can lead to *large oscillations* near the domain boundaries.

### What is Runge’s Phenomenon?

Runge’s phenomenon refers to the divergence of the interpolating polynomial from the true function as the degree increases, particularly when using equally spaced nodes.

### Instructions

1. Adjust the polynomial degree using the slider.
2. Observe how increasing the degree impacts interpolation quality.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sympy as sp

def runge_function(x):
    return 1 / (1 + 25 * x**2)

def interpolate_runge(degree):
    x = sp.symbols('x')
    domain = [-1, 1]

    x_nodes = np.linspace(domain[0], domain[1], degree + 1)
    y_nodes = runge_function(x_nodes)

    V = np.vander(x_nodes, degree + 1)
    coeffs = np.linalg.solve(V, y_nodes)

    p = np.poly1d(coeffs)

    x_plot = np.linspace(-1, 1, 1000)
    y_plot = runge_function(x_plot)
    y_poly = p(x_plot)

    fig, ax = plt.subplots(figsize=(10, 5))

    ax.plot(x_plot, y_plot, color="blue", lw=2, label="Runge's Function (Smooth Curve)")

    ax.plot(x_plot, y_poly, color="green", lw=2, linestyle="--", label=f"Degree {degree} Polynomial")

    ax.scatter(x_nodes, y_nodes, color='red', zorder=5, edgecolor='black', s=50, label="Interpolation Nodes")

    if degree <= 5:
        ax.text(-0.8, 0.8, "Good Approximation\nNo big oscillations", fontsize=10, color="black",
                bbox=dict(facecolor="lightgreen", edgecolor="black", boxstyle="round,pad=0.5"))
    elif degree <= 10:
        ax.text(-0.8, 0.8, "!! Some oscillations\nstart appearing\nnear edges", fontsize=10, color="black",
                bbox=dict(facecolor="yellow", edgecolor="black", boxstyle="round,pad=0.5"))
    else:
        ax.text(-0.8, 0.8, "Large oscillations!\nBad near edges\n(Runge's Phenomenon)", fontsize=10, color="black",
                bbox=dict(facecolor="red", edgecolor="black", boxstyle="round,pad=0.5"))

    if degree > 10:
        ax.axvspan(-1, -0.8, color="red", alpha=0.2)
        ax.axvspan(0.8, 1, color="red", alpha=0.2)
        ax.text(0.85, -0.6, "Edge oscillations!", fontsize=10, color="black", rotation=90,
        bbox=dict(facecolor="lightcoral", edgecolor="black", boxstyle="round,pad=0.5"))



    ax.set_title("Is Higher Degree Always Good?", fontsize=14, fontweight="bold")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.grid(True)
    ax.set_ylim(-2, 3)

    plt.show()

degree_slider = widgets.IntSlider(min=1, max=20, value=3, description="Degree:")
ui = widgets.VBox([degree_slider])
out = widgets.interactive_output(interpolate_runge, {'degree': degree_slider})

display(ui, out)


Output()

## Gibbs Phenomenon in Interpolation

In this section, we interpolate **discontinuous functions** using **Chebyshev nodes**. A canonical example is:

$$
f(x) =
\begin{cases}
1 & \text{if } x < 0 \\
0 & \text{if } x \geq 0
\end{cases}
$$

This kind of function introduces a **jump discontinuity**, which triggers the **Gibbs phenomenon**, an overshoot/undershoot near discontinuities.

### Gibbs Phenomenon

The interpolating polynomial cannot fully resolve the discontinuity and exhibits persistent oscillations near the jump, regardless of the polynomial degree.

### Instructions

1. Enter a piecewise function using `Piecewise` notation.
2. Adjust the degree and domain to observe how Gibbs phenomenon manifests.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sympy as sp
from IPython.display import display, Math

def interpolate_discontinuous(f_str, degree, domain):
    x = sp.symbols('x')

    try:
        f_expr = sp.sympify(f_str)
        f = sp.lambdify(x, f_expr, modules=['numpy'])
    except:
        print("Invalid function! Try again.")
        return

    n_points = degree + 1
    k = np.arange(n_points)
    cheb = np.cos((2 * k + 1) * np.pi / (2 * n_points))
    x_cheb = 0.5 * (domain[0] + domain[1]) + 0.5 * (domain[1] - domain[0]) * cheb
    y_cheb = f(x_cheb)

    V_cheb = np.vander(x_cheb, n_points)

    try:
        coeffs_cheb = np.linalg.solve(V_cheb, y_cheb)
    except np.linalg.LinAlgError:
        print("Singular matrix error - check function and degree!")
        return

    p_cheb = np.poly1d(coeffs_cheb)

    x_plot = np.linspace(domain[0], domain[1], 1000)
    y_plot = f(x_plot)
    y_poly_cheb = p_cheb(x_plot)

    error_cheb = np.abs(y_plot - y_poly_cheb)
    max_error_cheb = np.max(error_cheb)

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle("Interpolation for Discontinuous Function and Gibbs Phenomenon", fontsize=14, fontweight='bold')

    ax[0].plot(x_plot, y_plot, color="blue", lw=2, label="Original Function")
    ax[0].plot(x_plot, y_poly_cheb, color="green", linestyle="--", lw=2, label=f"Degree {degree} Polynomial")
    ax[0].scatter(x_cheb, y_cheb, color='red', edgecolor='black', s=50, label="Chebyshev Nodes")
    ax[0].set_title("Chebyshev Nodes - Interpolation")
    ax[0].legend()
    ax[0].grid(True)

    ax[1].plot(x_plot, error_cheb, color='purple', lw=2, label='Absolute Error')
    ax[1].set_title(f"Gibbs Phenomenon Error (Max: {max_error_cheb:.4e})")
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("|f(x) - P(x)|")
    ax[1].legend()
    ax[1].grid(True)

    plt.tight_layout()
    plt.show()

    poly_cheb_expr = sum(coeffs_cheb[i] * x**(degree - i) for i in range(n_points))
    display(Math(rf"\textbf{{Chebyshev Polynomial:}} \quad P_{{cheb}}(x) = {sp.latex(sp.expand(poly_cheb_expr))}"))

iw = widgets.interactive(interpolate_discontinuous,
                          f_str=widgets.Text(value='Piecewise((1, x < 0), (0, x >= 0))', description='Function:'),
                          degree=widgets.IntSlider(min=1, max=20, value=5, description='Degree:'),
                          domain=widgets.FloatRangeSlider(value=[-1, 1], min=-5, max=5, description='Domain:'))

display(iw)


interactive(children=(Text(value='Piecewise((1, x < 0), (0, x >= 0))', description='Function:'), IntSlider(val…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sympy as sp
from IPython.display import display, Math

def interpolation_error_plot(f_str, domain, max_degree=20, node_type='Equally Spaced'):
    x = sp.symbols('x')

    try:
        f_expr = sp.sympify(f_str)
        f = sp.lambdify(x, f_expr, modules=['numpy'])
    except:
        print("Invalid function!")
        return

    degrees = np.arange(1, max_degree + 1)
    max_errors = []
    rms_errors = []

    for degree in degrees:
        n_points = degree + 1

        if node_type == 'Equally Spaced':
            x_nodes = np.linspace(domain[0], domain[1], n_points)
        else:
            k = np.arange(n_points)
            cheb = np.cos((2 * k + 1) * np.pi / (2 * n_points))
            x_nodes = 0.5 * (domain[0] + domain[1]) + 0.5 * (domain[1] - domain[0]) * cheb

        y_nodes = f(x_nodes)

        V = np.vander(x_nodes, n_points)
        try:
            coeffs = np.linalg.solve(V, y_nodes)
        except np.linalg.LinAlgError:
            print(f"Singular matrix at degree {degree}. Stopping...")
            break

        p = np.poly1d(coeffs)

        x_test = np.linspace(domain[0], domain[1], 1000)
        y_true = f(x_test)
        y_poly = p(x_test)

        error = np.abs(y_true - y_poly)
        max_errors.append(np.max(error))
        rms_errors.append(np.sqrt(np.mean(error**2)))

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(degrees, max_errors, 'ro-', label="Max Error")
    ax.plot(degrees, rms_errors, 'bo-', label="RMS Error")
    ax.set_xlabel("Polynomial Degree")
    ax.set_ylabel("Error")
    ax.set_title(f"Error vs. Degree (Accuracy vs Complexity) ({node_type} Nodes)")
    ax.legend()
    ax.grid(True)
    plt.show()

iw = widgets.interactive(interpolation_error_plot,
                          f_str=widgets.Text(value='sin(x)', description='Function:'),
                          domain=widgets.FloatRangeSlider(value=[-3, 3], min=-10, max=10, description='Domain:'),
                          max_degree=widgets.IntSlider(min=1, max=30, value=15, description='Max Degree:'),
                          node_type=widgets.ToggleButtons(options=['Equally Spaced', 'Chebyshev'], description='Nodes:'))

display(iw)

interactive(children=(Text(value='sin(x)', description='Function:'), FloatRangeSlider(value=(-3.0, 3.0), descr…

## Cubic Spline Interpolation

This section introduces **cubic splines**, an alternative to global polynomial interpolation. A cubic spline is a **piecewise cubic polynomial** that ensures smoothness across intervals.

### Why Splines?

Global polynomials can oscillate wildly, especially for high degrees. Cubic splines provide local interpolation with:

- $\mathcal{C}^2$ continuity (2nd derivative continuous)
- Stable and accurate fitting for smooth and non-smooth data

Given $n$ data points $\{(x_i, y_i)\}$, the spline $S(x)$ satisfies:

- $S(x_i) = y_i$
- $S'(x)$ and $S''(x)$ are continuous at all internal nodes
- `natural` boundary conditions: $S''(x_0) = S''(x_n) = 0$

### Instructions

1. Enter a Python-compatible function.
2. Choose the number of interpolation points.
3. Set the domain of interpolation.

- The original function is plotted in blue.
- The spline interpolation is plotted in dashed green.
- Each subinterval is shaded to emphasize piecewise structure.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as si
import ipywidgets as widgets

def visualize_spline(f_str, num_points, domain):
    x = np.linspace(domain[0], domain[1], num_points)
    f = lambda x: eval(f_str, {"np": np, "x": x})
    y = f(x)

    spline = si.CubicSpline(x, y, bc_type='natural')

    x_fine = np.linspace(domain[0], domain[1], 1000)
    y_fine = f(x_fine)
    y_spline = spline(x_fine)

    fig, ax = plt.subplots(figsize=(10, 5))

    ax.plot(x_fine, y_fine, label="Original Function", lw=2, color="blue", alpha=0.6)

    ax.plot(x_fine, y_spline, '--', label="Cubic Spline Interpolation", lw=2, color="green")

    ax.scatter(x, y, color="red", s=50, label="Interpolation Nodes", zorder=5)

    for i in range(len(x) - 1):
        ax.axvspan(x[i], x[i + 1], color="lightgreen", alpha=0.15)

    ax.set_title("Cubic Splines (Piecewise Interpolation) - Smooth, Stable Interpolation", fontsize=14, fontweight="bold")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.grid(True)

    plt.show()

iw_spline = widgets.interactive(visualize_spline,
    f_str=widgets.Text(value='np.sin(x)', description='Function:'),
    num_points=widgets.IntSlider(min=3, max=30, value=10, description='Points:'),
    domain=widgets.FloatRangeSlider(value=[-7, 7], min=-10, max=10, description='Domain:')
)

display(iw_spline)


interactive(children=(Text(value='np.sin(x)', description='Function:'), IntSlider(value=10, description='Point…

## Least Squares Polynomial Approximation

When data is noisy or overdetermined (more points than polynomial degree + 1), exact interpolation is impossible or undesirable. Instead, **least squares polynomial fitting** finds coefficients $ \mathbf{c} $ minimizing

$$
\min_{\mathbf{c}} \sum_{i=1}^m (y_i - P_n(x_i))^2,
$$

where $ m $ is the number of data points, and $ P_n $ is polynomial of degree $ n $.

This problem is solved via the **normal equations** or QR decomposition using the Vandermonde matrix $ V $:

$$
\mathbf{c} = (V^T V)^{-1} V^T \mathbf{y}.
$$

### The below example

- Generates noisy samples of a base function.
- Uses least squares to fit polynomial of degree $ n $.
- Plots noisy data points, fitted polynomial, and true function.
- Displays fitting error.



- Higher-degree polynomials can overfit noise, leading to poor generalization.
- Lower-degree polynomials may underfit.
- Least squares fitting balances fitting error, robustness, and smoothness.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

def visualize_least_squares(f_str, degree, num_points, noise, domain):
    x = np.linspace(domain[0], domain[1], num_points)
    f = lambda x: eval(f_str, {"np": np, "x": x})
    y = f(x) + noise * np.random.randn(num_points)

    coeffs = np.polyfit(x, y, degree)
    p = np.poly1d(coeffs)

    x_fine = np.linspace(domain[0], domain[1], 1000)
    y_fine = f(x_fine)
    y_fit = p(x_fine)

    error = np.abs(y_fine - y_fit)

    fig, ax = plt.subplots(figsize=(10, 5))

    ax.plot(x_fine, y_fine, label="True Function (Ideal)", lw=2, color="blue", alpha=0.5)

    ax.plot(x_fine, y_fit, '--', label=f"Least Squares Fit (Degree {degree})", lw=2, color="green")

    ax.scatter(x, y, color="red", s=50, label="Noisy Data (Real-World)", zorder=5)

    ax.fill_between(x_fine, y_fine, y_fit, color="orange", alpha=0.3, label="Error Region")

    equation_str = "Fit: " + " + ".join([f"{coeff:.3f}x^{i}" for i, coeff in enumerate(coeffs[::-1])])
    equation_str = equation_str.replace("x^1 ", "x ")
    equation_str = equation_str.replace("x^0", "")

    ax.text(0.05, 0.9, equation_str, transform=ax.transAxes, fontsize=10,
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))

    ax.set_title("Least Squares Approximation + Fit Equation", fontsize=14, fontweight="bold")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.grid(True)

    plt.show()

iw_least_squares = widgets.interactive(visualize_least_squares,
    f_str=widgets.Text(value='np.sin(x)', description='Function:'),
    degree=widgets.IntSlider(min=1, max=15, value=3, description='Degree:'),
    num_points=widgets.IntSlider(min=5, max=50, value=20, description='Data Points:'),
    noise=widgets.FloatSlider(min=0, max=1, step=0.05, value=0.2, description='Noise Level:'),
    domain=widgets.FloatRangeSlider(value=[-3, 3], min=-10, max=10, description='Domain:')
)

display(iw_least_squares)


interactive(children=(Text(value='np.sin(x)', description='Function:'), IntSlider(value=3, description='Degree…